In [1]:
import pandas as pd
import numpy as np
import json
import os

In [8]:
# Define diretory name
directory = "episode_extracts_walk"

# initiate df and columns
transcripts_dataframe = pd.DataFrame()
all_showid = []
all_eposideid = []
all_transcripts = []
all_confidence = []
faulty_files = 0 # variable for checking how many faulty files

# iterate over files in dir
for dirpath, dirnames, files in os.walk(directory):
    for filename in files:
        transcripts = []
        confidence = []
        try:  # handle problem with files in wrng format (.DS_Store)
            with open(os.path.join(dirpath, filename)) as file:
                file_content = json.load(file)
                
                # index .json and loop at correct indent
                results = file_content["results"]
                for i in range(len(results)):
                    # lists are named after their key in the preceeding dict, dicts are named after their order
                    first_dict = results[i]
                    alternatives = first_dict["alternatives"]
                    second_dict = alternatives[0] 
                    if len(second_dict) == 3: # handle problem with empty alternative lists
                        transcripts.append(second_dict["transcript"])
                        confidence.append(second_dict["confidence"])
            
            all_showid.append(dirpath[-6:]) # OBS: change to [-27:] when running in Ucloud
            all_eposideid.append(str(filename))
            all_transcripts.append("".join(transcripts))
            all_confidence.append(np.mean(confidence))  # weighted confidence?
        
        except:
            faulty_files += 1

# Print errors message
print("Number of files with wrong format: ", faulty_files)

# Create dataframe
transcripts_dataframe["show_id"] = all_showid
transcripts_dataframe["episode_id"] = all_eposideid
transcripts_dataframe["transcript"] = all_transcripts
transcripts_dataframe["avg_confidence"] = all_confidence # should we do a weighted confidence?
transcripts_dataframe["word_count"] = transcripts_dataframe.transcript.apply(lambda x: len(x))

Number of files with wrong format:  7


In [9]:
# save new df as compressed csv
transcripts_dataframe.to_csv("transcripts_dataset.csv.gz", na_rep="Na", encoding="utf-8", compression="gzip", chunksize=5)
# print result
print(transcripts_dataframe.shape)
transcripts_dataframe.head(25)

(15, 5)


,show_id,episode_id,transcript,avg_confidence,word_count
0,show_0,7LOS9NNPOWOpsUynsjZupo.json,two Pura greetings to you listening around whe...,0.836709,2406
1,show_0,6qv1bIFOAF8COpdxfFTRgT.json,What is up guys? Welcome to a special edition ...,0.815195,50410
2,show_0,6F9ypi9sRnnM5XUtvcHhpL.json,You know what I did this morning around what's...,0.829023,55911
3,show_0,7sHyO8wLeEd1LuxfS8AIls.json,"Hey, hey. Hey. Hey. Hey, this is your girl Jen...",0.816693,60712
4,show_1,0JOymLFsRdeBVZbEA72ayj.json,Hello and welcome to the podcast the first eve...,0.807549,44940
5,show_1,0LM9rb9N4G501Z9vwgVqlC.json,Hello everybody and welcome to another edition...,0.816667,50372
6,show_1,1WosITIkpJemzZaPh8zAVb.json,This is the planetary potential podcast for th...,0.812145,19376
7,show_1,0rEooKQ89VcDG7vd006DE3.json,"Hello, you are now listening to The Stoke Hub ...",0.794734,26402
8,show_4,399kdfMnjw0KYANZU7CQJ0.json,It's the mother back a podcast. Well that was ...,0.813525,46848
9,show_3,56Cl2iY7KFRVS0QoMlS3u4.json,"Hello, you are now listening to The Stoke Hub ...",0.816775,39557
